In [22]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from pathlib import Path
from langchain.docstore.document import Document
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langgraph.graph import Graph
import torch 
load_dotenv()

True

In [23]:
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


In [24]:
class Utf8TextLoader(TextLoader):
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        with open(self.file_path, 'r', encoding='utf-8') as f:
            return f.read()


def read_text_file(file_path):
    return Path(file_path).read_text(encoding='utf-8')


def create_documents_from_files(file_paths):
    documents = []
    for file_path in file_paths:
        content = read_text_file(file_path)
        documents.append(Document(page_content=content))
    return documents

In [25]:
file_paths = list(Path('./source').glob("*.txt"))
docs = create_documents_from_files(file_paths)

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

In [27]:
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]
print(doc_strings)

['El deporte es una actividad física que desempeña un papel crucial en la vida de las personas en todo el mundo. No solo es una fuente de entretenimiento y competencia, sino que también promueve la salud física y mental, fomenta el trabajo en equipo y contribuye al desarrollo de valores sociales y culturales. La diversidad de disciplinas deportivas, desde el fútbol y el baloncesto hasta el atletismo', 'y el baloncesto hasta el atletismo y la natación, ofrece opciones para todos los gustos y habilidades, haciendo del deporte una parte integral de la sociedad moderna.', 'Uno de los deportes más populares a nivel global es el fútbol. Conocido como "el deporte rey", el fútbol se juega y se sigue fervientemente en todos los continentes. La Copa Mundial de la FIFA, celebrada cada cuatro años, es uno de los eventos deportivos más vistos en el planeta, uniendo a millones de aficionados en una celebración de habilidad, estrategia y pasión. Las ligas de fútbol de todo el', 'pasión. Las ligas de 

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [29]:
print(f"Using device: {device}")

Using device: cpu


In [30]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
embeddings.__dict__

d:\Proyecto_holagrama_IA\LanGraph\v2email\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'client': SentenceTransformer(
   (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
   (2): Normalize()
 ),
 'model_name': 'sentence-transformers/all-mpnet-base-v2',
 'cache_folder': None,
 'model_kwargs': {'device': 'cpu'},
 'encode_kwargs': {'normalize_embeddings': True},
 'multi_process': False,
 'show_progress': False}

In [31]:
db = Chroma.from_documents(new_docs, embeddings)
db

In [32]:
retriever = db.as_retriever(search_kwargs={"k": 4})
retriever.__dict__

{'name': None,
 'tags': ['Chroma', 'HuggingFaceEmbeddings'],
 'metadata': None,
 'vectorstore': <langchain_community.vectorstores.chroma.Chroma at 0x29fb7523970>,
 'search_type': 'similarity',
 'search_kwargs': {'k': 4}}

In [33]:
query = "Cuentame sobre la cultura japonesa"
docs = retriever.invoke(query)
print(docs)

[Document(page_content='La cultura japonesa es un fascinante contraste entre la tradición y la modernidad. Por un lado, el país es famoso por sus antiguas prácticas y costumbres, como el arte del té, la ceremonia del kimono, y los jardines zen. Lugares emblemáticos como el Templo Senso-ji en Tokio, el Santuario Fushimi Inari-taisha en Kioto y el Castillo de Himeji son testamentos de su herencia histórica y'), Document(page_content='La cultura japonesa es un fascinante contraste entre la tradición y la modernidad. Por un lado, el país es famoso por sus antiguas prácticas y costumbres, como el arte del té, la ceremonia del kimono, y los jardines zen. Lugares emblemáticos como el Templo Senso-ji en Tokio, el Santuario Fushimi Inari-taisha en Kioto y el Castillo de Himeji son testamentos de su herencia histórica y'), Document(page_content='En términos de educación y ética laboral, Japón es conocido por su riguroso sistema educativo y su cultura de trabajo dedicada. La disciplina y el respe

En extensión .py funciona bien y da el formato bien

In [34]:
AgentState = {}
AgentState["messages"] = []

In [35]:
AgentState

{'messages': []}

In [36]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192", streaming=False)

In [37]:
def function_1(state):
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    
    complete_query = "Su tarea es proporcionar sólo el tema basado en la consulta del usuario. \
        Sólo se emite el tema entre: [Japan , Sports]. No incluya el razonamiento. A continuación se muestra la consulta del usuario: " + question
    response = llm.invoke(complete_query)
    state['messages'].append(response.content) # appending LLM call response to the AgentState
    return state

In [38]:
def function_2(state):
    messages = state['messages']
    question = messages[0] 

    template = """Responda a la pregunta basándose únicamente en el siguiente contexto:
    {context}

    Pregunta: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return result

In [39]:
workflow = Graph()

workflow.add_node("Agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('Agent', 'tool')

workflow.set_entry_point("Agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [40]:
inputs = {"messages": ["Háblame del crecimiento industrial de Japón"]}
app.invoke(inputs)

['Háblame del crecimiento industrial de Japón', 'Japan']


'Según el texto, Japón es un líder mundial en tecnología e innovación. Ciudades como Tokio y Osaka son centros neurálgicos de avances tecnológicos, y son hogar de gigantes de la industria como Sony, Toyota y Nintendo. Esto sugiere que Japón ha experimentado un crecimiento industrial significativo, especialmente en la tecnología y la innovación.'

In [41]:
inputs = {"messages": ["¿El deporte reduce el riesgo de enfermedades crónicas como la diabetes?"]}
app.invoke(inputs)

['¿El deporte reduce el riesgo de enfermedades crónicas como la diabetes?', 'Sports']


'Sí, según el texto, el deporte reduce el riesgo de enfermedades crónicas como la diabetes.'

In [42]:
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Salida del Nodo '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Salida del Nodo 'Agent':
---
{'messages': ['¿El deporte reduce el riesgo de enfermedades crónicas como la diabetes?', 'Sports', 'Sports']}

---

['¿El deporte reduce el riesgo de enfermedades crónicas como la diabetes?', 'Sports', 'Sports']
Salida del Nodo 'tool':
---
Sí, según el texto, el deporte reduce el riesgo de enfermedades crónicas como la diabetes.

---

